In [ ]:
#from __future__ import (absolute_import, division, print_function, unicode_literals)
#import absolute_import

import pywt 
import urllib.request, urllib.error, urllib.parse
import json
import numpy as np
import statsmodels.tsa.stattools as stats
import statsmodels.api as sm
import pandas as pd
import pprint
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta as dlta
#import tensorflow as tf
import matplotlib.pyplot as plot
import skfuzzy as fuzz
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import time
from prices import fullprices

#import numpy as np
#import matplotlib.pyplot as plt
#import pycwt as wavelet
#from pycwt.helpers import find
from matplotlib.image import NonUniformImage

from pandas.stats.api import ols

class PairsTrader(object):
    def __init__(self):
        self.df = {}#pd.DataFrame()
        #self.periods = [12,1,2,3,4,5,6,9,24,48]#months
        self.mvadays = 10
        #note: notebook saves in current directory (paths are not relative)
        #self.hdf_path = "pairs.h5"
        self.coints = []
        self.limits = [.93,1.069,.931,1.07] # [buyTight,unTighten,unLoosen,buyLoose] i.e. [.94,.99,1.01,1.06] conservative [.94,1.05,.95,1.06] aggressive
        self.cash = 1000.00 # starting cash
        self.debug = False
        self.confs = ["1%","5%","10%"] # confidence intervals we want, when we run Augmented Dicky-Fuller
        self.startDate="2016-05-01"
        self.endDate="2017-04-28"
        self.stocks = ['CTSH','ADP','PYPL','YHOO','AVGO','EBAY','GLW','MU','VMW','PSX','APC','HAL','WMB','VLO','MPC','BHI','SE','DVN','APA','CB','MMC','BBT','CME','STT','EQR','AFL','ALL','DFS','BEN','AAL','EMR','CSX','DE','ITW','ETN','NSC','WM','CMI','SYK','BDX','HUM','VRTX','CAH','HCA','ILMN','BAX','MYL','BXLT','TSLA','CBS','CCL','LVS','FOX','TRI','DISH','VIA','MDLZ','COST','CL','KHC','KMB','RAI','KR','GIS','ADM','EL']           
        self.prices = []
        self.pairs = []    
        self.corr = .95
        self.wavelet_corr=.85 #using waveletes to smooth price and test correlation
        self.results=[]
        
    def getPairs(self):
        self.prices=fullprices()
        print(len(self.prices),len(self.stocks))
        wv_corr=0.0
        #YAHOO API having network issues, stored all these prices in prices.py
#        for ticker in self.stocks:
#            print("ticker: ",ticker)
#            time.sleep(1)
#            tickerPrices = y.Share(ticker).get_historical(self.startDate,self.endDate)
#            cleanPrices =[]
#            for j in tickerPrices: cleanPrices.append(float(j['Adj_Close']))
#            if self.debug: print("cleanPrices: ",cleanPrices)
#            self.prices.append(cleanPrices[::-1])#reverse prices, most recent at end of list
#            print(self.prices)
        print("len: ",len(self.prices[0]))
        j=1
        for i in range(len(self.stocks)):
            for j in range(i+1,len(self.stocks)):
                try:#catching the diff lengths exception
                    #wv_corr=self.waveletCalc([self.prices[i],self.prices[j]])
                    if self.debug: print("wv_corr: ",wv_corr)
                    if(np.corrcoef(self.prices[i],self.prices[j])[1,0] > self.corr):# and wv_corr>self.wavelet_corr:
                        print("corr found: ",self.stocks[i],self.stocks[j])
                        self.pairs.append([self.stocks[i],self.stocks[j]])
                except:""
        print("pairs: ",self.pairs,"num Pairs: ",len(self.pairs))
              
    def runPairs(self,fuzzy=True):
        for i in self.pairs:
            prc1=self.prices[self.stocks.index(i[0])]
            prc2=self.prices[self.stocks.index(i[1])]
            if self.debug: print("prc1: ",prc1)
            if self.debug: print("prc2: ",prc2)
            tic1,tic2=i[0],i[1]
            if prc2[0]>prc1[0]: 
                prc1,prc2=prc2,prc1#keep higher price on top, logic in tradePairs expects this
                tic1,tic2=tic2,tic1
            self.tradePairs(prc1,prc2,tic1,tic2,fuzzy)
    
    def tradePairs(self,prc1,prc2,tic1,tic2,fuzzy=True):
        # stor averages, look for deviations away from average,
        # expect cointegration relationship to pull them back to normal
        buyLong=buyShort=sellLong=sellShort=profit=highLoose=totBuys=totSells=buyDiff=chProfit=mProfit=0.0
        lowTight=100.0
        cash = self.cash
        df = self.df
        days = self.mvadays
        limits = self.limits
        
        perceptron([prc1,prc2])
        
        gc1,gc2=self.gc([prc1,prc2])
        bla1,bla2=self.bla([prc1,prc2])
        fuzz1=self.fuzzy(gc1,bla1)
        fuzz2=self.fuzzy(gc2,bla2)
        
        #calculating percentages to put in leader and follower stocks
        #note: 100.0-leadernumber represents we are putting the majority in the follower stocks
        perc1=1.0-fuzz1*(1.0/(fuzz1+fuzz2))
        perc2=1.0-fuzz2*(1.0/(fuzz1+fuzz2))
        
        #flag to turn off fuzzy logic, to compare results
        if not fuzzy:
            perc1=perc2=.5 #even distro, normal pairs trading
            
        print("fuzz1,fuzz2: ",fuzz1,fuzz2)
        print("perc1,perc2: ",perc1,perc2)
        
        #playing around with leader/follower percentages
        if not fuzzy:
            perc1=perc2=.5 #even distro, normal pairs trading
        #perc1,perc2=1.0,0.0 #all first stock
        #perc1,perc2=0.0,1.0 #all second stock
        
        #self.wavCohere([prc1,prc2])
        
        if self.debug: print("prc1: ",prc1)
        if self.debug: print("prc2: ",prc2)
        
        if prc1.__len__()!=prc2.__len__():
            print("len mismatch")
            return
        
        #display vars
        dmvas=[]
        cashbals=[]
        buysellsigs=[] # to show on the graph if we're long or short
        
        tight=loose=lastDay=False
        for i in range(prc1.__len__()):
            if i==len(prc1)-1: lastDay=False#True #Taking out last day logic for now, simply make True to cash out on last day
            if i<days:
              dmvas.append(0)
              cashbals.append(cash/100.0)
              buysellsigs.append(5)#neutral
              continue#days needed to calc moving average
            mmavg=np.array(prc1[i-days:i]).mean()
            cmavg=np.array(prc2[i-days:i]).mean()
            dmavg=mmavg-cmavg
            dmvas.append(dmavg)
            cashbals.append(cash/100.0)
            diff=prc1[i]-prc2[i]
            
            if self.debug: print("prc1,mmavg,prc2,cmavg,diff,dmavg,dratio: ",prc1[i],mmavg,prc2[i],cmavg,(prc1[i]-prc2[i]),dmavg,diff/dmavg,buyDiff,limits[1]*dmavg)
            
            if(diff/dmavg < lowTight):lowTight=diff/dmavg
            if(diff/dmavg > highLoose):highLoose=diff/dmavg
            
            #detect a tight relationship, long the upper price and short the lower price
            #cannot be in tight or loose relationship to execute
            if((not tight and not loose) and diff<limits[0]*dmavg):
                if self.debug: print("buying tight",diff/dmavg)
                tight=True
                buyLong=prc1[i]
                sellShort=prc2[i]
                
                if self.debug: print("Long prc1 at: ",prc1[i])
                if self.debug: print("Short prc2 at: ",prc2[i])
                
                buyDiff=diff
                totBuys+=1
                buysellsigs.append(6)
            #detect a loose relationship, short the upper price and long the lower price
            #cannot be in tight or loose relationship to execute
            elif((not loose and not tight) and diff>limits[3]*dmavg):
                if self.debug: print("buying loose",diff/dmavg)
                loose=True
                sellShort=prc1[i]
                buyLong=prc2[i]
                
                if self.debug: print("Short prc1 at: ",prc1[i])
                if self.debug: print("Long prc2 at: ",prc2[i])
                
                buyDiff=diff
                totBuys+=1
                buysellsigs.append(4)
            #in tight, look for a sell signal 
            elif(tight and diff>limits[1]*dmavg) or (lastDay and tight):
                if lastDay: print("lastDay, untightening")
                if self.debug: print("untighten",diff/dmavg)
                tight = False
                sellLong=prc1[i]
                buyShort=prc2[i]
                mProfit+=sellLong-buyLong
                chProfit+=sellShort-buyShort
                
                #handling cash
                if self.debug: print("cash before: ",cash)
                cash = ((cash*perc1) * (sellLong/buyLong)) + ((cash*perc2) * (sellShort/buyShort))
                #cash = cash * sellLong/buyLong #all money in prc1
                if self.debug: print("prc1 buy,sell prices,delta: ",buyLong,sellLong,sellLong-buyLong)
                if self.debug: print("prc2 buy,sell prices,delta: ",buyShort,sellShort,sellShort-buyShort)
                if self.debug: print("new cash: ",cash)
                
                buyLong=buyShort=sellLong=sellShort=0.0
                totSells+=1
                buysellsigs.append(5)
            #in loose, look for a sell signal
            elif(loose and diff<limits[2]*dmavg) or (lastDay and loose):
                if lastDay: print("lastDay, unloosening")
                if self.debug: print("unloose",diff/dmavg)
                loose = False
                buyShort=prc1[i]
                sellLong=prc2[i]
                mProfit+=sellShort-buyShort
                chProfit+=sellLong-buyLong
                
                #handling cash
                if self.debug: print("cash before: ",cash)
                cash = ((cash*perc2) * (sellLong/buyLong)) + ((cash*perc1) * (sellShort/buyShort))
                #cash = cash * sellShort/buyShort #all money in prc1
                if self.debug: print("prc2 buy,sell prices,delta: ",buyLong,sellLong,sellLong-buyLong)
                if self.debug: print("prc1 buy,sell prices,delta: ",buyShort,sellShort,sellShort-buyShort)
                if self.debug: print("new cash: ",cash)
                
                buyLong=buyShort=sellLong=sellShort=0.0
                totSells+=1
                buysellsigs.append(5)
            #no buy/sell triggers this cycle
            elif(tight):
                if self.debug: print("Still tight, waiting to untighten",diff/dmavg)
                buysellsigs.append(6)
            elif(loose):
                if self.debug: print("Still loose, waiting to unloosen",diff/dmavg)
                buysellsigs.append(4)
            else:
                if self.debug: print("All quiet",diff/dmavg)
                buysellsigs.append(5)
        print("\n",tic1,"-",tic2,"Pairs trading results...")
        print("highLoose: ",highLoose)
        print("lowTight: ",lowTight)
        print("total buys: ",totBuys)
        print("total sells: ",totSells)
        print(tic1," profit%: ",mProfit/np.mean(prc1))
        print(tic2," profit%: ",chProfit/np.mean(prc2))
        #print("total profit: ",chProfit+mProfit)
        print("Note: Final percentages will not match change in cash, because each stock is allocated a different percentage of captial.  See perc1,perc2 above")
        print("Beginning cash: ",self.cash)
        print("Final cash: ",cash)
        
        plt=plot
        plt.plot(prc1, label=str(tic1+" price"), linewidth=5.0)
        plt.plot(prc2, label=str(tic2+" price"), linewidth=5.0)
        plt.plot(cashbals, label=str("cash balance"), linewidth=5.0)
        plt.plot(dmvas,label=str(str(self.mvadays)+" day moving average"), linewidth=5.0)
        plt.plot(buysellsigs,label=str("long or short"), linewidth=5.0)
        #plt.title(str(pairs[0] + " " + pairs[1] + " Deltas"),fontsize=20.0)
        #plt.xticks([2,19],[self.startDate,self.endDate])
        plt.legend()
        plt.show()
        self.results.append([tic1,tic2,cash/self.cash,gc1,gc2,bla1,bla2,fuzz1,fuzz2,perc1,perc2])
   
    def finalResults(self):
        return self.results
    
    #This class uses Google Tensorflow to test for cointegration
    #tensorflow will create a single level neural net, where each perceptron is checking for cointegration for the given time interval
    '''
    def runTensorflow(self):
        df = self.df
        periods = self.periods
        cadfs=[]
        coints=[]
        
        sess = tf.Session()
        
        #creating perceptrons - one for each cointegration test
        #above we specify we are testing for prc1/prc2 prices for : 1M,2M,3M,4M,5M,6M,9M and 12M
        for i in periods:
          ms = df["prc1"+str(i)]
          cs = df["prc2"+str(i)]
          
          #Creating and running perceptron(ms,cs)
          #Note: the perceptron code is located outside this class, because it is serialized
          #Since Python classes are also serialized this avoids serializing twice
          inp = tf.constant([ms,cs])
          y = tf.py_func(perceptron, [inp], tf.float32)
          print("sess: ", sess.run(y))
          coints.append(sess.run(y))
          
        self.coints = coints
        return coints
    '''
    
    def testCoints(self):
        pass
    
    def adfs(self):
        pass
    
    
        
    def gc(self,x):
        if len(x[0]) != len(x[1]): 
            print("error, causality test must be same size arrays!")
            return (-1.0,-1.0)
        
        #convert two arrays to an array of pairs of tuples
        gtest=[]
        gtest2=[]
        for i in range(len(x[0])):
            gtest.append((x[0][i],x[1][i]))#gtest is done with an array of tuples
            gtest2.append((x[1][i],x[0][i]))#reverse to test gc the other direction
        if self.debug: print("gtest: ",gtest)
        if self.debug: print("gtest2: ",gtest2)
        
        #running test, here we go!!!
        s = stats.grangercausalitytests(gtest,5)#5 is numLags
        s2 = stats.grangercausalitytests(gtest2,5)
        if self.debug: print("s: ",s)
        if self.debug: print("s2: ",s2)
        
        #getting lowest p-score for all lags, p-score less than 0.05 allows us to reject null of no causality
        #test, just pulling first lag, usually the lowest
        gc1=s[1][0]['params_ftest'][1]#,s[2]['params_ftest'][0],s[3]['params_ftest'][0],s[4]['params_ftest'][0],s[5]['params_ftest'][0])#max of 5 lags of gcs
        gc2=s2[1][0]['params_ftest'][1]#,s2[2]['params_ftest'][0],s2[3]['params_ftest'][0],s2[4]['params_ftest'][0],s2[5]['params_ftest'][0])#max of 5 lags of gcs
        
        #pulling lowest of 5 lags, p-scores (usually its the first lag, but not always)
        gc1=min(s[1][0]['params_ftest'][1],s[2][0]['params_ftest'][1],s[3][0]['params_ftest'][1],s[4][0]['params_ftest'][1],s[5][0]['params_ftest'][1])#max of 5 lags of gcs
        gc2=min(s2[1][0]['params_ftest'][1],s2[2][0]['params_ftest'][1],s2[3][0]['params_ftest'][1],s2[4][0]['params_ftest'][1],s2[5][0]['params_ftest'][1])#max of 5 lags of gcs
        
        print("gc1: ",gc1)
        print("gc2: ",gc2)
        return gc1,gc2
    
    def waveletCalc(self,x):
        print("wave test")
        #discrete wavelet transform
        wv1,wv2=x[0],x[1]
        wv1_dwt_ca,wv1_dwt_cd = pywt.dwt(wv1,'db2')#haar, morl
        wv2_dwt_ca,wv2_dwt_cd = pywt.dwt(wv2,'db2')#haar, morl
        print("wv after")
        #modwt (), stationary wavelet transform
        #obtaining approximation coefficients (similar to values of wavelet) and details coefficients, (changes)
        
        db2 = pywt.Wavelet('db2')#daubechies 2 
        ca_cd1= pywt.swt(wv1, db2, level=1,start_level=0)
        ca_cd2= pywt.swt(wv2, db2, level=1,start_level=0)
        wv1_ca,wv1_cd=ca_cd1[0][0],ca_cd1[0][1]
        wv2_ca,wv2_cd=ca_cd2[0][0],ca_cd2[0][1]
        
        wv_corr=np.corrcoef(wv1,wv2)[1,0]
        swt_ca_corr=np.corrcoef(wv1_ca,wv2_ca)[1,0]
        swt_cd_corr=np.corrcoef(wv1_cd,wv2_cd)[1,0]
        print("corr Orig Wavelets: ",wv_corr)
        print("corr SWT approx Coeffs: ",swt_ca_corr)
        print("corr SWT details Coeffs: ",swt_cd_corr)
        
        return swt_cd_corr
    
    def bla(self,x):#brim leadership ratio
        p1,p2=x[0],x[1]
        diff = 0.0
        diffs = []
        #diffavg = np.abs((np.sum(p1) - np.sum(p2)) / p1.__len__()) * 1.0
        diffavg = 0.0
        move = 0.0
        p1move = 0.0
        p2move = 0.0
        p1moves = []
        p2moves = []
        p1diff = 0.0
        p2diff = 0.0

        if ((p1.__len__() != p2.__len__()) or (p1.__len__() < 6) or (p2.__len__() < 6)):
            #print "Array Lengths must match and be great than 6 and exact same size"
            return 0.0,0.0,0.0,0.0
        brimfuzzy = 0
        brimfuzzy2 = 0
        for i in range(5,p1.__len__()):
            ###print "i: ",i
            diffavg = np.abs((np.sum(p1[i-5:i-1]) / 5.0) - (np.sum(p2[i-5:i-1]) / 5.0))
            ##print "diffavg: ",diffavg
            diff = np.abs(p1[i] - p2[i]) * 1.0
            ##print "diff: ",diff
            diffs.append(diff)
            move = np.abs(diff - diffavg) * 1.0
            ##print "move: ", move
            p1diff = np.abs(p1[i] - p1[i-1]) * 1.0
            p2diff = np.abs(p2[i] - p2[i-1]) * 1.0
            ##print "p1diff: ",p1diff
            ##print "p2diff: ",p2diff
            if (p1diff == 0.0): p1move=0.0
            else: p1move = move / (p1diff * 1.0)
            if (p2diff == 0.0): p2move=0.0
            else: p2move = move / (p2diff * 1.0)
            ##print "p1move: ",p1move
            ##print "p2move: ",p2move
            p1moves.append(p1move)
            p2moves.append(p2move)
        ##print "\n\np1: ",p1
        ##print "p2: ",p2
        ##print "diffs: ",diffs
        ##print "p1moves: ",p1moves
        ##print "p2moves: ",p2moves
        p1leadscore = (np.sum(p1moves) / p1.__len__())
        p2leadscore = (np.sum(p2moves) / p2.__len__())
        p1leadratio = (np.sum(p1moves) / p1.__len__()) / (np.sum(p2moves) / p2.__len__())
        p2leadratio = (np.sum(p2moves) / p2.__len__()) / (np.sum(p1moves) / p1.__len__())
        if(p1leadratio < 1.1 and p1leadratio > .9):#FIXME, no movements could be detected as leader
            brimfuzzy+=1
        if(p2leadratio < 1.05 and p2leadratio > .95):
            brimfuzzy2+=1

        print("p1lead,p2lead,p1ratio,p2ratio: ",p1leadscore,p2leadscore,p1leadratio,p2leadratio)

        #return p1leadscore,p2leadscore,p1leadratio,p2leadratio,brimfuzzy,brimfuzzy2
        return p1leadratio,p2leadratio
    
    #wavelet coherence, 
    #cross wavelet transform
    def wavCohere(self,x):
        s1,s2=x[0],x[1]
        dt=1.0
        W12, cross_coi, freq, signif = wavelet.xwt(s1, s2, dt, dj=1/12, s0=-1, J=-1,significance_level=0.8646, wavelet='morlet',normalize=True)
    
    def fuzzy(self,gc=.001,bla=1.2):
    
        #scale gc and bla to 1-10 scale
        newgc=0.0
        newbla=0.0
        
        if(gc>.09): 
            newgc=0.0
        elif(gc<0.0):
            newgc=0.0
        else:
            newgc=(0.1-gc)*100
                 
        if(bla>2.0):
            newbla=10.0
        else:
            newbla=bla*5.0
                  
        print("gc,newgc: ",gc,newgc)
        print("bla,newbla: ",bla,newbla)
        
        #gc lo 0.0-0.05, med 0.5-1.0, high 1.0-infinity
        #bla  lo med ..5- or 1.1-1.2, high med .9-1.1,
        
        # Generate universe variables
        #   * gcity and blaice on subjective ranges [0, 10]
        #   * lead has a range of [0, 25] in units of percentage points
        x_gc = np.arange(0, 11, 1)
        x_bla = np.arange(0, 11, 1)
        x_lead  = np.arange(0, 11, 1)
        
        # Generate fuzzy membership functions
        gc_lo = fuzz.trimf(x_gc, [0, 0, 3])
        gc_md = fuzz.trimf(x_gc, [3, 5, 7])
        gc_hi = fuzz.trimf(x_gc, [7, 10, 10])
        bla_lo = fuzz.trimf(x_bla, [0, 0, 3])
        bla_md = fuzz.trimf(x_bla, [3, 5, 7])
        bla_hi = fuzz.trimf(x_bla, [7, 10, 10])
        lead_lo = fuzz.trimf(x_lead, [0, 0, 3])
        lead_md = fuzz.trimf(x_lead, [3, 5, 7])
        lead_hi = fuzz.trimf(x_lead, [7, 10, 10])
        
        #Visualize1
        
        # We need the activation of our fuzzy membership functions at these values.
        # The exact values 6.5 and 9.8 do not exist on our universes...
        # This is what fuzz.interp_membership exists for!3
        # it doesn't matter that 6.5 is the same for lo md hi, it just cant be a number in the definition set above
        
        gc_level_lo = fuzz.interp_membership(x_gc, gc_lo, newgc)
        gc_level_md = fuzz.interp_membership(x_gc, gc_md, newgc)
        gc_level_hi = fuzz.interp_membership(x_gc, gc_hi, newgc)
        
        bla_level_lo = fuzz.interp_membership(x_bla, bla_lo, newbla)
        bla_level_md = fuzz.interp_membership(x_bla, bla_md, newbla)
        bla_level_hi = fuzz.interp_membership(x_bla, bla_hi, newbla)
        
        # Now we take our rules and apply them. Rule 1 concerns bad food OR blaice.
        # The OR operator means we take the maximum of these two.
        active_rule1 = np.fmax(gc_level_lo, bla_level_lo)
        
        # Now we apply this by clipping the top off the corresponding output
        # membership function with `np.fmin`
        lead_activation_lo = np.fmin(active_rule1, lead_lo)  # removed entirely to 0
        
        # For rule 2 we connect acceptable blaice to medium leadping
        lead_activation_md = np.fmin(bla_level_md, lead_md)
        
        # For rule 3 we connect high blaice OR high food with high leadping
        active_rule3 = np.fmax(gc_level_hi, bla_level_hi)
        lead_activation_hi = np.fmin(active_rule3, lead_hi)
        lead0 = np.zeros_like(x_lead)
        #
        
        #Visualize2
        
        # Aggregate all three output membership functions together
        aggregated = np.fmax(lead_activation_lo,
                             np.fmax(lead_activation_md, lead_activation_hi))
        
        # Calculate defuzzified result
        lead = fuzz.defuzz(x_lead, aggregated, 'centroid')
        lead_activation = fuzz.interp_membership(x_lead, aggregated, lead)  # for plot
        print("lead: ",lead)
        return lead
    
    def saveHdf(self):
        pass
        #need to clean this up self.df.to_hdf(self.hdf_path,'random',mode='a')5
        
def perceptron(x):
    # print "perceptron prc1: ",x[0]
    # print "perceptron prc2: ",x[1]
  
    confs=["1%","5%","10%"]
    pdf = pd.DataFrame()
    pdf["prc1"]=x[0]
    pdf["prc2"]=x[1]
    
    res = ols(x=pdf["prc1"],y=pdf["prc2"])
    print("res: ",res)
    beta_hr = res.beta.x
    pdf["res"] = pdf["prc1"] - beta_hr*pdf["prc2"]

    print("beta_hr: ",beta_hr)
    #print "pdf.res: ",pdf["res"]
    
    adf = ts.adfuller(pdf["res"])
    pprint.pprint(adf)
    #cadfs.append(adf)
    print("cadf: ",adf)
    coints=[]
    for key in confs:coints.append(np.float32(adf[4][key]))
      
    print("perceptron coints: ",coints)

    #self.cadfs = cadfs
    return [coints]
    #return x
  

    
def main():
    trader = PairsTrader()
    #trader.buildData()
    trader.getPairs()
    trader.runPairs()
    trader.runPairs(False)
    print(trader.finalResults())
    #caedfs = trader.runTensorflow()
    #pprint.pprint(cadfs)
    #trader.trade()
    trader.saveHdf()

if __name__=="__main__":main()
    